In [3]:
import numpy as np
import random
import tensorflow as tf
import pandas as pd
import cv2


In [4]:
seed_constant = 42
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [5]:


# Load your Excel dataset
df = pd.read_excel("D:/Desktop/Internship/Research Paper/Robot dataset/meta_data.xlsx")

# Make sure episode_id is a string to match the folder name
df['episode_id'] = df['episode_id'].astype(str)

In [6]:
import os

# Define frame root directory
base_path = r"D:\Desktop\Internship\Research Paper\Robot dataset\save"
positive_path = os.path.join(base_path, "Positive")
negative_path = os.path.join(base_path, "Negative")


In [7]:
# ✅ Step 3: Function to extract episode ID from folder name
import re

def extract_episode_id(folder_name):
    """
    Extracts the numeric episode ID from folder names like:
    'manual#split_train#episode_9#_0' → '9'
    """
    match = re.search(r"episode_(\d+)", folder_name)
    if match:
        return str(int(match.group(1)))  # remove leading zeros
    return None

In [8]:
# ✅ Step 4: Create label map from Excel before scanning folders
df['episode_id'] = df['episode_id'].astype(str)
df['instruction'] = df['instruction'].astype(str)
df['ground_truth_narration'] = df['ground_truth_narration'].astype(str)
df['undesired_behaviors'] = df['undesired_behaviors'].astype(str)

# ✅ Create a dictionary mapping episode_id → all relevant labels as dictionary
label_map = {
    row['episode_id']: {
        'binary_label': row['binary_label'],
        'instruction': row['instruction'],
        'ground_truth_narration': row['ground_truth_narration'],
        'undesired_behaviors': row['undesired_behaviors']  
    }
    for _, row in df.iterrows() 
}

# ✅ Step 5: Scan folders and collect info
folder_info = []

for label_folder, label_value in [("Positive", 1), ("Negative", 0)]:
    class_path = os.path.join(base_path, label_folder)
    if not os.path.exists(class_path):
        continue

    for folder_name in os.listdir(class_path):
        folder_path = os.path.join(class_path, folder_name)

        if os.path.isdir(folder_path):
            episode_id = extract_episode_id(folder_name)

            if episode_id is not None:
                episode_data = label_map.get(str(episode_id))

                if episode_data:
                    folder_info.append({
                        "folder_name": folder_name,
                        "episode_id": int(episode_id),
                        "binary_label": episode_data['binary_label'],
                        "instruction": episode_data['instruction'],
                        "ground_truth_narration": episode_data['ground_truth_narration'],
                        "undesired_behaviors": episode_data['undesired_behaviors'],
                        "target_variable": label_folder,
                    
                    })
                else:
                    print(f"⚠️ Episode ID {episode_id} not found in Excel.")
            else:
                print(f"⚠️ Could not extract episode_id from: {folder_name}")


In [9]:
# ✅ Convert to DataFrame and sort by episode_id (numerically)
df_info = pd.DataFrame(folder_info)

In [10]:
# Make sure episode_id is treated as integer for correct sorting
df_info['episode_id'] = df_info['episode_id'].astype(int)

In [11]:
# Sort by episode_id from 0 to 174 or 175
df_info = df_info.sort_values(by='episode_id').reset_index(drop=True)

In [12]:
# ✅ Display in notebook
from IPython.display import display
display(df_info)


,folder_name,episode_id,binary_label,instruction,ground_truth_narration,undesired_behaviors,target_variable
0,manual#split_train#episode_0#_0,0,1,pick up a bag of chips from the table,t=0: the robot's gripper approaches the bag of...,[] the robot squeezes the middle of the packag...,Positive
1,manual#split_train#episode_2#_0,2,1,pick up the carrot and put it on the plate,t=0: the robot's gripper approaches the carrot...,[] the robot’s gripper presses the fragile pla...,Positive
2,manual#split_train#episode_5#_0,5,1,pick up a bag of chips from the table,t=0: the robot's gripper approaches the bag of...,[] the robot squeezes the middle of the packag...,Positive
3,manual#split_train#episode_6#_0,6,1,pick up a bag of chips from the table,t=0: the robot's gripper approaches the bag of...,[] the robot briefly hesitates before picking ...,Positive
4,manual#split_train#episode_7#_0,7,1,add some coke into the glass,t=0: the robot's gripper firmly grasps an open...,"[] while serving coke to the guests, the robot...",Positive
...,...,...,...,...,...,...,...
168,manual#split_train#episode_171#_0,171,1,putting the spoon into the bowl of soup,t=0: The robot's gripper holds a spoon and app...,[],Positive
169,manual#split_train#episode_172#_0,172,1,pick up the bread on the table (you can assume...,t=0: the robot's gripper approaches the bread\...,[] while attempting to pick up the bread from ...,Positive
170,manual#split_train#episode_173#_0,173,1,pick up the bread on the table (you can assume...,t=0: the robot's gripper approaches the bread\...,[] while attempting to pick up the bread from ...,Positive
171,manual#split_train#episode_174#_0,174,1,pick up the bread on the table (you can assume...,t=0: The robot's gripper approaches the bread ...,[],Positive


In [13]:
display(df_info.tail(10))  # Display first 10 rows for quick check

,folder_name,episode_id,binary_label,instruction,ground_truth_narration,undesired_behaviors,target_variable
163,manual#split_train#episode_166#_0,166,0,pick up the ceramic bowl and put in the small ...,t=0: The robot's gripper reaches toward the ce...,[] the robot clumsily dropped the fragile cera...,Negative
164,manual#split_train#episode_167#_0,167,1,move the turner and place it at the left edge ...,t=0: The robot's gripper approaches the spatul...,[] the robot hit the pot when picking up the t...,Positive
165,manual#split_train#episode_168#_0,168,0,put potato into the vessel,t=0: The robot's gripper approaches a potato\n...,[] the robot dropped the potato at an inapprop...,Negative
166,manual#split_train#episode_169#_0,169,1,move the pot to the left edge of the table,t=0: The robot's gripper reaches for a pot whi...,[] the robot caused a collision between the po...,Positive
167,manual#split_train#episode_170#_0,170,0,put broccoli in pot,t=0: The robot's gripper approaches an assortm...,[] the robot dropped the broccoli at an inappr...,Negative
168,manual#split_train#episode_171#_0,171,1,putting the spoon into the bowl of soup,t=0: The robot's gripper holds a spoon and app...,[],Positive
169,manual#split_train#episode_172#_0,172,1,pick up the bread on the table (you can assume...,t=0: the robot's gripper approaches the bread\...,[] while attempting to pick up the bread from ...,Positive
170,manual#split_train#episode_173#_0,173,1,pick up the bread on the table (you can assume...,t=0: the robot's gripper approaches the bread\...,[] while attempting to pick up the bread from ...,Positive
171,manual#split_train#episode_174#_0,174,1,pick up the bread on the table (you can assume...,t=0: The robot's gripper approaches the bread ...,[],Positive
172,manual#split_train#episode_175#_0,175,1,putting a container of orange juice infront of...,"t=0: A person sits at a table, waiting \nt=1:...",[],Positive


In [14]:
# Image loading and preprocessing
def load_images_from_folder(folder_path, target_size=(224, 224)):
    images = []
    for filename in sorted(os.listdir(folder_path)):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            path = os.path.join(folder_path, filename)
            img = cv2.imread(path)
            if img is not None:
                img = cv2.resize(img, target_size)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = img.astype('float32') / 255.0
                images.append(img)
    return np.array(images)


In [15]:
# Load sequences per episode
X_sequences = []
y_sequences = []
episode_ids = []

# 🔧 CHANGE: Added validation for sequence length
print("Loading sequences...")
valid_sequences = 0
invalid_sequences = 0

for _, row in df_info.iterrows():
    folder_name = row['folder_name']
    target_variable = row['target_variable']
    episode_id = row['episode_id']
    binary_label = row['binary_label']

    folder_path = os.path.join(base_path, target_variable, folder_name)
    if os.path.exists(folder_path):
        images = load_images_from_folder(folder_path)
        # 🔧 CHANGE: Added validation and warning for frame count
        if len(images) == 20:  # ensure all videos have 20 frames
            X_sequences.append(images)
            y_sequences.append(binary_label)
            episode_ids.append(episode_id)
            valid_sequences += 1
        else:
            print(f"⚠️ Episode {episode_id} has {len(images)} frames instead of 20. Skipping.")
            invalid_sequences += 1
    else:
        print(f"⚠️ Folder not found: {folder_path}")

print(f"✅ Loaded {valid_sequences} valid sequences")
print(f"⚠️ Skipped {invalid_sequences} invalid sequences")

X_sequences = np.array(X_sequences)         # Shape: (N, 20, 224, 224, 3)
y_sequences = np.array(y_sequences)
episode_ids = np.array(episode_ids)

print(f"Final data shape: {X_sequences.shape}")

Loading sequences...
✅ Loaded 173 valid sequences
⚠️ Skipped 0 invalid sequences
Final data shape: (173, 20, 224, 224, 3)


In [16]:
from sklearn.model_selection import train_test_split

# Split episode IDs to ensure no data leakage
unique_episodes = np.unique(episode_ids)
unique_labels = [y_sequences[np.where(episode_ids == ep)[0][0]] for ep in unique_episodes]

# Split episodes, not individual sequences
train_episodes, test_episodes = train_test_split(
    unique_episodes, 
    test_size=0.3, 
    random_state=seed_constant,
    stratify=unique_labels
)

print(f"Train episodes: {len(train_episodes)}")
print(f"Test episodes: {len(test_episodes)}")

# Create masks for splitting sequences
train_mask = np.isin(episode_ids, train_episodes)
test_mask = np.isin(episode_ids, test_episodes)

X_train_seq = X_sequences[train_mask]
y_train_seq = y_sequences[train_mask]
X_test_seq = X_sequences[test_mask]
y_test_seq = y_sequences[test_mask]

Train episodes: 121
Test episodes: 52


In [17]:
# Print info
print("✅ Sequence Data Ready")
print("Train shape:", X_train_seq.shape)
print("Test shape:", X_test_seq.shape)
print("Train label distribution:", np.bincount(y_train_seq))
print("Test label distribution:", np.bincount(y_test_seq))


✅ Sequence Data Ready
Train shape: (121, 20, 224, 224, 3)
Test shape: (52, 20, 224, 224, 3)
Train label distribution: [40 81]
Test label distribution: [17 35]


In [23]:
import tensorflow as tf
from tensorflow.keras.layers import TimeDistributed, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping


CLASSES_LIST = ['Positive', 'Negative']

def create_convlstm_model():
    cnn = tf.keras.models.Sequential()

    # 1st ConvLSTM layer
    print("Adding 1st ConvLSTM layer...")
    cnn.add(tf.keras.layers.ConvLSTM2D(
        filters=32, kernel_size=(3, 3), 
        data_format="channels_last", 
        activation='tanh', 
        return_sequences=True, 
        recurrent_dropout=0.2, 
        input_shape=(20, 224, 224, 3)
    ))
    cnn.add(tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2), data_format="channels_last", padding="same"))
    cnn.add(TimeDistributed(BatchNormalization()))
    print(f"After 1st block: {cnn.output_shape}")

    # 2nd ConvLSTM layer
    print("Adding 2nd ConvLSTM layer...")
    cnn.add(tf.keras.layers.ConvLSTM2D(
        filters=64, kernel_size=(3,3), 
        activation='tanh', 
        return_sequences=True, 
        recurrent_dropout=0.2, 
        data_format="channels_last"
    ))
    cnn.add(tf.keras.layers.MaxPooling3D(pool_size=(1,2,2), data_format="channels_last", padding="same"))
    cnn.add(TimeDistributed(BatchNormalization()))
    print(f"After 2nd block: {cnn.output_shape}")

    # 3rd ConvLSTM layer
    print("Adding 3rd ConvLSTM layer...")
    cnn.add(tf.keras.layers.ConvLSTM2D(
        filters=128, kernel_size=(3,3), 
        activation="tanh", 
        data_format="channels_last", 
        return_sequences=True, 
        recurrent_dropout=0.2
    ))
    cnn.add(tf.keras.layers.MaxPooling3D(pool_size=(1,2,2), data_format="channels_last", padding="same"))
    cnn.add(TimeDistributed(BatchNormalization()))
    print(f"After 3rd block: {cnn.output_shape}")

    # 4th ConvLSTM layer - CRITICAL CHANGE
    print("Adding 4th ConvLSTM layer with return_sequences=False...")
    cnn.add(tf.keras.layers.ConvLSTM2D(
        filters=256, kernel_size=(3,3), 
        data_format="channels_last", 
        activation="tanh", 
        return_sequences=False,  # This should make it 4D
        recurrent_dropout=0.2
    ))
    print(f"After 4th ConvLSTM (return_sequences=False): {cnn.output_shape}")
    
    # Add 2D operations for 4D tensor
    print("Adding MaxPooling2D...")
    cnn.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding="same"))
    print(f"After MaxPooling2D: {cnn.output_shape}")
    
    print("Adding BatchNormalization...")
    cnn.add(tf.keras.layers.BatchNormalization())
    print(f"After BatchNorm: {cnn.output_shape}")

    # Dropout layer
    print("Adding Dropout...")
    cnn.add(tf.keras.layers.Dropout(0.5))
    print(f"After Dropout: {cnn.output_shape}")

    # 🔧 FLATTEN LAYER
    print("Adding Flatten layer...")
    cnn.add(tf.keras.layers.Flatten())
    print(f"After Flatten: {cnn.output_shape}")

    # 🔧 DENSE LAYERS
    print("Adding Dense layer...")
    cnn.add(tf.keras.layers.Dense(128, activation='relu'))
    print(f"After Dense(128): {cnn.output_shape}")
    
    cnn.add(tf.keras.layers.Dropout(0.5))
    print(f"After Dropout: {cnn.output_shape}")

    print("Adding Output Dense layer...")
    cnn.add(tf.keras.layers.Dense(1, activation="sigmoid"))
    print(f"After Output Dense: {cnn.output_shape}")


    early_stopping_callback= EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        mode='min'  # 'min' for loss, 'max' for accuracy
    )
    # Compile model
    cnn.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )
    

    print("Showing model summary...")
    cnn.summary()
    
    return cnn



In [24]:
# Constructing the required ConvLSTM model
convlstm_model = create_convlstm_model()

print("✅ ConvLSTM Model Created")

Adding 1st ConvLSTM layer...
After 1st block: (None, 20, 111, 111, 32)
Adding 2nd ConvLSTM layer...
After 2nd block: (None, 20, 55, 55, 64)
Adding 3rd ConvLSTM layer...
After 3rd block: (None, 20, 27, 27, 128)
Adding 4th ConvLSTM layer with return_sequences=False...
After 4th ConvLSTM (return_sequences=False): (None, 25, 25, 256)
Adding MaxPooling2D...
After MaxPooling2D: (None, 13, 13, 256)
Adding BatchNormalization...
After BatchNorm: (None, 13, 13, 256)
Adding Dropout...
After Dropout: (None, 13, 13, 256)
Adding Flatten layer...
After Flatten: (None, 43264)
Adding Dense layer...
After Dense(128): (None, 128)
After Dropout: (None, 128)
Adding Output Dense layer...
After Output Dense: (None, 1)
Showing model summary...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d_4 (ConvLSTM2D)  (None, 20, 222, 222, 32   40448     
                             )                    

In [25]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    mode='min'
)

# Train the model
convlstm_model_training = convlstm_model.fit(
    x=X_train_seq,
    y=y_train_seq,
    validation_data=(X_test_seq, y_test_seq),
    epochs=50,
    batch_size=8,
    shuffle=True,
    callbacks=[early_stopping_callback],
    verbose=1
)

Epoch 1/50


: 